##### [시카고 샌드위치 맛집 분석]
- <학습목표>
- 웹 데이터 가져오는 Beautiful Soup
- 개발자도구 F12  -> copy 사용
- 시카고 샌드위치 맛집 소개 사이트에 접근 => error 남
- 다수의 웹페이지에서 자동으로 접근해서 원하는 정보 가져오기
- Jupyter notebook 상태 진행바 생성
- 상태 진행바 적용 페이지 접근하기
- 50개 웹페이지 정보 가져오기


#### [requests module] -> 데이터 수집에 최적화된 라이브러리(get())
- APP ---요청--> Web Site
-        get()
-       post()
-     delete()
-     patch()

- get: select (데이터 수집)
- post: Insert -> put()
-       update -> patch()
-       delete  -> delete()

#### get()
- 1.url: 요청 보낼 대상 url
- 2.param(get 요청시 사용) -> 로그인 정보, Query String방식
-  딕셔너리방식으로 요청하면 Query string 방식으로 받는다
- 3.data(post,put 일때 사용)->딕셔너리:{'id':'hong','pw':'1234'} : 대부분 로그인은 post 방식 사용(get사용X)

#### 결론: get()은 파라미터 써야함

- headers -> 받는쪽 서버에서 봇 인식 여부 확인
- timeout(): 요청 대기 시간을 초단위 지정

#### response object나옴

-response object : 속성(스수집한 데이터,처리상태) / 메소드()\
-1.속성(Attribute)
-   text: 서버가 응답한 내용을 문자열(데이터)
-  content:서버가 응답한 내용을 Byte로 반환(이진수) -> 이미지,동영상,PDF,엑셀
-          "이미지 수집은 content로!!"
-   json: {key:value} ----> 시간 응답 서버
-       ex.)LS 증권 API (웹브라우저 이용X)
- status.code:응답상태 정보 [응답상태정보 확인하면서 코딩해야함]
-       200:정상 응답
-       403:권한 X
-       404:요청하는 page가 없다.(요청 URL페이지가 없다.) / 400번대 나오면 조심해야함
-       500:서버쪽 에러



In [2]:
## 코드 시작
import requests
import pandas as pd
import numpy as np

In [3]:
url = 'https://www.google.com'
response = requests.get(url)

response

if response.status_code == 200 :
    print('요청 성공!')
    print(response.text[:100]) #100자만 확인
else:
    print(f'요청 실패 :{response.status.code}')
    
# 요청실패되면 요쳥실패만 출력하고 끝남
# 

요청 성공!
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content


In [4]:
# 예외 처리 방식 권장
url = 'https://www.naver.com'

# 상태에 따른 오류 코드

while(True): # While 넣는 순간, 받을때까지 계속 돌리기
    try:
        response = requests.get(url, timeout = 5)
        # get 은 트라이 할 이유가 없음
        # 400,500 에러 발생시 예외 발생
        response.raise_for_status() #400,500일때 raise 할 수 있음    
        # 데이터 처리
        print(response.text[0:100])
        break # 죽을때까지 돌리기....DDOS 공격인줄 알음 
              # 200일땐 정상적일땐 출력하고 끝
              # 400-> break 안걸림 -> 무한반복 즉 ,에러나면 계속 반복
              # <정리> 정상:반복 X , 오류:계속 반복
        
    # 에러 순서도 기억하기
    except requests.exception.HTTPError as errh: #프로토콜 에러
        print(f'Http Error : {errh}')

    except requests.exception.ConnectionError as errc : # 연결 에러
        print(f'Connection Error : {errc}')

    except requests.exception.Timeout as errt:
        print(f'Timeout Error : {errt}') # 

    except requests.exception.RequestException as err:
        print(f'Something Error : {err}')


   <!doctype html> <html lang="ko" class="fzoom"> <head> <meta charset="utf-8"> <meta name="Referrer


In [5]:
# 네이버 검색
url = 'https://search.naver.com/search.naver'
# ?where=nexearch&=python
params = {
    'where' : 'nexearch',
    'query' : '어쩔수가없다' # 원하는 검색어 넣기
}
response = requests.get(
    url,
    params = params
)
print(f"요청된 URL : {response.url}") # 실제 요청된 전체 URL 확인
print(response.text[:100])

#https://httpbin.org/ : 내가 만든 코드가 잘 보내지고 받는지 검증하는 사이트


요청된 URL : https://search.naver.com/search.naver?where=nexearch&query=%EC%96%B4%EC%A9%94%EC%88%98%EA%B0%80%EC%97%86%EB%8B%A4
<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="strict-


In [6]:
# post test
url = 'https://httpbin.org/post'

# 잘하면 naver뚫을 수도 있다. ->naver -blog, cafe 수집가능함


In [7]:
# form data 전송 post test
form_data = {
    'username' :'myname',
    'password' :'1234'
}

response = requests.post(url, data=form_data)

print(response.json()) #jason 으로 하는이유: 서버가 주는 방식대로 하는거임~
# 결과: 'form': {'password': '1234', 'username': 'myname'} 나오면 잘 갔다 온거임~~


{'args': {}, 'data': '', 'files': {}, 'form': {'password': '1234', 'username': 'myname'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '29', 'Content-Type': 'application/x-www-form-urlencoded', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.32.5', 'X-Amzn-Trace-Id': 'Root=1-68f989a6-099c4925378bca331c65df44'}, 'json': None, 'origin': '183.98.149.174', 'url': 'https://httpbin.org/post'}


In [8]:
# json data 전송 post test
url = 'https://httpbin.org/post'

# 전송할 딕셔너리 데이터
dict_data = {
    'custid':'1000',
    'prodid':'200',
    'tags' : ['elec','com']
}

response = requests.post(url, json = dict_data)

print(response.json()['json']) # 우리가 보내는 키를 json으로 확인할 수 있다. -> 데이터 수집하는데 도움 많이 됨


{'custid': '1000', 'prodid': '200', 'tags': ['elec', 'com']}


In [9]:
url = 'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36' # 개발자 도구 ->네트워크->요청 해더 -> user agent ,크롬웹브라우저랑 똑같은 형태로 붙이기 됨
    #우리사이트가 웹 또는 모바일 어느겅세 접근 많은지 확인
    
}

response = requests.get(url,headers=headers) # headers 가 들어가야 로봇이 아닌지 확인 됨
print(response.text[:100])

#결과 :403 Forbidde - 파이썬 코드다!하고 막은거임


<!DOCTYPE html>
<html lang="en-US">
<head>
	<meta charset="UTF-8">
	<meta http-equiv="X-UA-Compatibl


In [10]:
# 20251023
# urlopen(url)->파일다운로드
# 데이터 수집 모듈 1.requests[일반적] 2. urlopen(url)

# 1.requests : post 방식
# <post 방식> 데이터수집할때 로그인해서 들어가야함
# 근데 로그인 왜 해야지?

#### 사람이 로그인하는 목적 (순서-강의내용 참고)
- main화면 말고 다음작업을 하기 위해서
- request,response : 둘다 header/body로 구성됨
- session쿠키(key)생성
- 자물쇠 + 키를 세션쿠키에 담아서 클라이언트한테 보낸댜.
- 문제)키가넘어가긴하지만 request+response 하지만 좌물쇠를 지워버린다 : 비연결성
- 서버입장에서는 " 난 너 모르겠다" -> 비 연결성 , http 자체가 비연결성 지향한다.
- 결국,한번요쳥하고 그 다음에는  "서버曰 난 너 몰라~~~" 
- (참고: 웹사이트에서 session id라는 키 확인 가능)


- 해결: 서버쪽에서 좌물쇠를 저장해놔야함~~(기억해놓아야지 나중에 모르는척 방지~!!)
- 이 관리: Session
- session 을 다루지 못하면 다음 단계로 나가지 못함~~

- application ,server에서도 모두 session 을 다루는 넘이 있어야함
- requests.Session() : 생성  ----요청------> server


- <정리> reqest 에서 세선 유지목적 2가지
- 1. 로그인상태 유지 -> 이후작업 계속 진행
- 2. 만약 세션안하면 서버쪽에서는 계속 키를 생성하기때문에 시간, 메모리 오버 생김 -> 세션처리해야함
- 2_1.여러페이지 요청할 경우 세션이용하면 속도향상


#### HTTP -> 비연결성(한번요청하면 연결 끊김)
- clinet  ---request----> server ----response---> clinet 하면 한번하고 끊김
- requests 다 객체임, 새로운 reuqest 만들고 날려버림 -> 연결유지가 안되는거임'
- 'ㅋㅣ'를 가지고 이전에 요쳥했던 놈인지 아닌지를 파악함
- <중요> request는 어딘가에 저장해 논다고? -> session

In [14]:
#### 시카고 사이트는 header가 안되기에 url X, request 로 요청해야함

# with 문을 사용해 세션 생성
with requests.Session() as session:
    # 세션을 사용한 요청 처리
    response = session.get('https://httpbin.org/get')
    print(response.text)


{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.32.5", 
    "X-Amzn-Trace-Id": "Root=1-68f98ca7-644803c27ed066df6ade77bb"
  }, 
  "origin": "183.98.149.174", 
  "url": "https://httpbin.org/get"
}



In [ ]:
#### beautifulsoup: 가져온 데이터 추출 -> 한쌍으로 추출